1. Version  : 1
2. Time     : 5th May
3. Author   : Hui Zhang
4. Function : PBCD method to solve 2 hidden layer 0/1 DNN on Fashion MNIST
5. Structure: 2 hidden layer with 2000 nodes in each layer
6. Relation paper: PBCD method for 0/1 DNN

In [11]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount("/content/drive/", force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Define relavant function and load data**

In [12]:
'''
    Function: 10 Classification for fashion
    Version: Feb18th-version**
    Network: 2 hidden layer with 2000 neurons, 0/1 actativation + hardmax
    Time: 2022.02.18
'''

import numpy as np   # science calculate  instill numpy
import struct  # y???????
from sklearn.utils import shuffle  # instill scikit-learn/ scikit
import time  
import os  # o??????????
import scipy
import torch as tc
import tensorflow as tf
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

from tqdm import tqdm, trange
import tensorflow.keras as keras
from tensorflow.keras import datasets
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense,Conv2D,Dropout,Flatten,Activation,BatchNormalization,AveragePooling2D,MaxPooling2D
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Standardized data in 0-1
def normalize_data(ima):
    a_max = np.max(ima)  # 图像矩阵的最大值
    a_min = np.min(ima)  # 图像矩阵的最小值
    for i in range(ima.shape[0]):  # 矩阵第二维的长度：如3*4矩阵这个就是3
        for j in range(ima.shape[1]):  # numpy.core.fromnumeric 中的函数 第一维的长度
            ima[i][j] = (ima[i][j] - a_min) / (a_max - a_min)
    return ima  # 返回是一个矩阵

# 初始化各个参数 n_x x的维数 n_h1：第一隐层节点数（靠近x的）, n_h2：第二隐层节点数, n_y： 输出y的维数
def initialize_with_zeros(n_x, n_h1, n_h2, n_y):
    tf.random.set_seed(21)
    W1 = tf.random.uniform([n_x, n_h1], minval = -np.sqrt(6) / np.sqrt(n_x + n_h1), maxval = np.sqrt(6) / np.sqrt(n_h1 + n_x), dtype=tf.dtypes.float32, seed=None, name=None)
    # b1 = tf.zeros((n_h1, 1))
    # 可选取不同的 gamma1 = np.random.uniform(low=-0.5, high=0.5, size=(n_x, 1))
    # gamma1 = tf.ones((n_x, 1))  # np.random.randn(n_x, 1)

    W2 = tf.random.uniform([n_h1, n_h2], minval = -np.sqrt(6) / np.sqrt(n_h1 + n_h2), maxval = np.sqrt(6) / np.sqrt(n_h1 + n_h2), dtype=tf.dtypes.float32, seed=None, name=None)
    # gamma2 = tf.ones((n_h1, 1))  #

    W3 = tf.random.uniform([n_h2, n_y], minval = -np.sqrt(6) / np.sqrt(n_y + n_h2), maxval = np.sqrt(6) / np.sqrt(n_y + n_h2), dtype=tf.dtypes.float32, seed=None, name=None)
    # b3 = tf.zeros((n_y, 1))
    # gamma3 = tf.ones((n_h2, 1))  # 

    dW1_1 = tf.zeros((n_x, n_h1))
    dW1_2 = tf.zeros((n_x, n_h1))
    dW2_1 = tf.zeros((n_h1, n_h2))
    dW2_2 = tf.zeros((n_h1, n_h2))
    dW3_1 = tf.zeros((n_h2, n_y))
    dW3_2 = tf.zeros((n_h2, n_y))

    g_sum = {"dW1_1": dW1_1,
             "dW1_2": dW1_2,
             "dW2_1": dW2_1,
             "dW2_2": dW2_2,
             "dW3_1": dW3_1,
             "dW3_2": dW3_2
             }
    parameters = {"W1": W1,
                  "W2": W2,
                  "W3": W3,
                  "idx1": np.ones((n_x, 1)),  # initial
                  "idx2": np.ones((n_h1, 1)),  # initial
                  "idx3": np.ones((n_h2, 1))  # initial
                  }
    pa_in = 1.e-7 * np.ones((4, 1))  # 每一层罚参数初始值
    pa_in[3] = 1 / (2 * batch_size)  # 每一层罚参数初始值
    tau_in = 1.e-6 * np.ones((4, 1))  # 每一层罚参数初始值
    return parameters, pa_in, tau_in, g_sum

def costloss(V3, Y):
    cost = tf.square(tf.norm(V3 - Y)) / 2
    return cost

def penaltyloss(P, C, X, Y):
    cost1 = pa[3] * tf.square( tf.norm(Y - hardmax(C["U3"])) ) 
    cost2 = tau[3] * tf.square(tf.norm( C["U3"] - tf.matmul(tf.transpose(P['W3']),  C["V2"]))  )
    cost3 = pa[2] * tf.square( tf.norm(C["V2"] - sgn(C["U2"])) ) + tau[2] * tf.square( tf.norm(C["U2"] - tf.matmul(tf.transpose(P['W2']), C["V1"])) )
    print("Cheack out, now-------------------")
    print(pa[2], (tf.square( tf.norm(C["V2"] - sgn(C["U2"])) )).numpy(), tau[2], (tf.square( tf.norm(C["U2"] - tf.matmul(tf.transpose(P['W2']), C["V1"])) )).numpy())
    print("------------------------------------")
    cost4 =  pa[1] * tf.square( tf.norm(C["V1"] - sgn(C["U1"])) ) + tau[1] * tf.square( tf.norm(C["U1"] - tf.matmul(tf.transpose(P['W1']), X)) )
    cost5 = gam * (tf.square( tf.norm(P['W3'])) + tf.square(tf.norm(P['W2'])) + tf.square(tf.norm(P['W1'])))
    
    cost = {"cost1": cost1.numpy(),
             "cost2": cost2.numpy(),
             "cost3": cost3.numpy(),
             "cost4": cost4.numpy(),
             "cost5": cost5.numpy(),
             "costall": cost1.numpy() + cost2.numpy() + cost3.numpy()+cost4.numpy()+cost5.numpy()
             }
    return cost

def sgn(x):
    s = tf.sign(x * tf.cast(x > 0, dtype=tf.float32))
    return s


def softmax(x):  # 返回第一个最大值的位置
    v = tf.argmax(x)
    return v

def hardmax(x):  # 返回矩阵值 dimision: 10 * 64
    v1 = np.zeros(x.shape)
    v = np.argmax(x, 0)  # tf.argmax(a,1) 每一列最大值位置
    for i in range(x.shape[1]):
        v1[v[i], i] = 1

    return tf.convert_to_tensor(v1, dtype=tf.float32)

# 将原矩阵拉成784列但不知道多少行（-1）的矩阵（np.会自动计算配套维数）
def image2vector(image):
    v = np.reshape(image, [-1, n_x])
    return v.T


def mini_batchs_generator(inputs, targets, batch_size):
    inputs_data_size = len(inputs)
    targets_data_size = len(targets)
    # print("len(inputs):={}, size of inputs {}, size of targets{}".format(inputs_data_size, inputs.shape, targets.shape))
    assert inputs_data_size == targets_data_size, "The length of inputs({}) and targets({}) must be consistent".format(
        inputs_data_size, targets_data_size)

    shuffled_input, shuffled_target = shuffle(inputs, targets)  # 元素随机排序？
    mini_batches = [(shuffled_input[k: k + batch_size], shuffled_target[k: k + batch_size])
                    for k in range(0, inputs_data_size, batch_size)]  #
    for x, y in mini_batches:
        # print("~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        # print(x.shape)
        yield x, y  # yield = return的功能 + 未终止再使用直到循环完毕(实现一个epoch功能)



def produce_step_adam(grad_1, grad_2, g_new, epsilon, delta, rho_1, rho_2, step_ad):
    """
    adam
     (i) 梯度滑动-一阶矩  g = rho_1 * grad_1 + (1-rho_1)*g_new
     (ii)二阶矩  r_t = rho_2 * grad + (1-rho_1)*g_new*g_new
     (iii)更新  step = epsilon /sqrt(delta+r_t)
    """
    grad_1 = rho_1 * grad_1 + (1 - rho_1) * g_new
    g = grad_1 / (1 - pow(rho_1, step_ad))
    r_t = rho_2 * grad_2 + (1 - rho_2) * g_new * g_new
    r = r_t / (1 - pow(rho_2, step_ad))
    step = epsilon / tf.sqrt(delta + r)
    return step, g, grad_1, r_t


def prox_l0_pre(x, sparse, vec_yn):
    k = int((x.shape[0] * x.shape[1]) * 0.5) if vec_yn else int(x.shape[0] * 0.51)
    if vec_yn:
        if k > 5:
            abs_x = tf.abs(x)
            thre0 = np.partition(abs_x, kth=k, axis=None)[k]
            thre = tf.convert_to_tensor(thre0, dtype=tf.float32)
            x *= (abs_x >= thre)
    else:
        x = tf.linalg.norm(x, axis=1, keepdims=True)
        # print(np.partition(x, kth=k, axis=None)[k], np.sqrt(2 * 0.13 * sparse * lr))
        x *= x >= tf.sqrt(2 * sparse * lr)
        # if k > 5:
        #     thre = np.partition(x, kth=k, axis=None)[k]
        #     x *= (x >= thre)
    return x


def forward_propagation_pre(x_tol, parameters_for):
    layers_num = 3
    cache_for = {}
    layer_input = x_tol
    for layer in range(1, layers_num + 1):
        locals()['A' + str(layer)] = tf.matmul(tf.transpose(parameters_for['W' + str(layer)]),
                                            layer_input)
        cache_for['A' + str(layer)] = locals()['A' + str(layer)]
        locals()['Z' + str(layer)] = locals()['A' + str(layer)] * tf.cast(locals()['A' + str(layer)] > 0, dtype=tf.float32)
        layer_input = locals()['Z' + str(layer)]
        cache_for['Z' + str(layer)] = locals()['Z' + str(layer)]
    return locals()['Z' + str(layers_num)], cache_for


def optimizer_adam_l0(parameters, cache, X, Y, g_sum, epsilon, delta, sp0, rho_1, rho_2, step_ad, iht=False):
    A1 = cache["Z1"]
    A2 = cache["Z2"]
    A3 = cache["Z3"]

    dZ3 = (A3 - Y)  # * Z3(1-Z3)  # h2 * batch
    dZ2 = tf.matmul(parameters["W3"], dZ3) * tf.cast(A2 > 0,dtype=tf.float32)
    dZ1 = tf.matmul(parameters["W2"], dZ2) * tf.cast(A1 > 0,dtype=tf.float32)

    dW3 = tf.matmul((A2), tf.transpose(dZ3))
    dW2 = tf.matmul((A1), tf.transpose(dZ2))
    dW1 = tf.matmul((X), tf.transpose(dZ1))

    step_W3, dW3, g_sum["dW3_1"], g_sum["dW3_2"] = produce_step_adam(g_sum["dW3_1"], g_sum["dW3_2"], dW3,
                                                                     epsilon, delta, rho_1, rho_2, step_ad)
    step_W2, dW2, g_sum["dW2_1"], g_sum["dW2_2"] = produce_step_adam(g_sum["dW2_1"], g_sum["dW2_2"], dW2,
                                                                     epsilon, delta, rho_1, rho_2, step_ad)
    step_W1, dW1, g_sum["dW1_1"], g_sum["dW1_2"] = produce_step_adam(g_sum["dW1_1"], g_sum["dW1_2"], dW1, epsilon,
                                                                     delta, rho_1, rho_2, step_ad)

    parameters["W3"] -= step_W3 * dW3
    parameters["W2"] -= step_W2 * dW2
    parameters["W1"] -= step_W1 * dW1

    if iht:
        parameters["W3"] = prox_l0_pre(parameters["W2"], sp0, vec_yn=1)
        parameters["W2"] = prox_l0_pre(parameters["W2"], sp0, vec_yn=1)
        parameters["W1"] = prox_l0_pre(parameters["W1"], sp0, vec_yn=1)

    return parameters, g_sum


# feed forward
def forward_propagation(X, parameters):
    W1 = parameters["W1"]  # W1 维数 （784, 1000）
    W2 = parameters["W2"]
    W3 = parameters["W3"]

    U1 = tf.matmul(tf.transpose(W1), X)  # + b1 * parameters['gamma1']
    # betch_size =16, nx*nx = 784 X为（784，16）, gamma1为nx*nx，X * gamma1的维数和X维数相同
    V1 = sgn(U1)  # 实现Relu，V1为 (1000，16)为第1隐层的值
    U2 = tf.matmul(tf.transpose(W2), V1)  # + b2    # W2^T 维数 （1000,1000） * parameters['gamma2']
    V2 = sgn(U2)  # 实现Relu，V2 (1000，16)为第2隐层的值
    U3 = tf.matmul(tf.transpose(W3), V2)  # + b3    # W3^T为 （10，1000） * parameters['gamma3']
    V3 = hardmax(U3)  # 输出 维数是10*batch 大小

    cache = {
        "V0": X,
        "U0": X,
        "U1": U1,
        "V1": V1,
        "U2": U2,
        "V2": V2,
        "U3": U3,
        "V3": V3}

    return V3, cache


def prox_l0(x, sparse, vec_yn):
    k = int(x.shape[0] * sp0)
    if vec_yn:
        if k > 5:
            abs_x = tf.abs(x)
            thre = np.partition(abs_x, kth=k, axis=None)[k]
            x *= (abs_x >= thre)
    else:
        abx = np.linalg.norm(x, axis=1, keepdims=True)
        # print(abx.shape, x.shape[0], k)
        thre = np.partition(abx, kth=k, axis=None)[k]
        abx *= (abx >= thre)
        # print(np.partition(x, kth=k, axis=None)[k], np.sqrt(2 * 0.13 * sparse * lr))
        # x *= x >= np.sqrt(2 * sparse * lr)
        # if k > 5:
        #     thre = np.partition(x, kth=k, axis=None)[k]
        #     x *= (x >= thre)
    return abx

# 存储参数
def save_parameters(parameters, directory):
    for key, val in parameters.items():
        if not os.path.exists(os.path.join(directory, str(key) + '.npy')):
          os.makedirs(os.path.join(directory, str(key) + '.npy'))
        np.save(os.path.join(directory, str(key) + '.npy'), val)

def load_parameters(directoryin):
    W1 = np.load(os.path.join(directoryin, 'W1.npy'))
    W2 = np.load(os.path.join(directoryin, 'W2.npy'))
    W3 = np.load(os.path.join(directoryin, 'W3.npy'))
    parameters = {"idx1": np.nonzero(np.ones((W1.shape[0], 1)))[0],
                  "idx2": np.nonzero(np.ones((W2.shape[0], 1)))[0],  
                  "idx3": np.nonzero(np.ones((W3.shape[0], 1)))[0],
                  "W1": tf.convert_to_tensor(W1, dtype=tf.float32),
                  "W2": tf.convert_to_tensor(W2, dtype=tf.float32),
                  "W3": tf.convert_to_tensor(W3, dtype=tf.float32)
                  }
    return parameters


if __name__ == '__main__':
    data_name = "fashion10"
    path_dict = {1: "01_BCD"}
    directory = ("/content/drive/MyDrive/PBCDcode/FashionMNIST/data/")
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # ============Load dataset
    train_images = np.load(os.path.join(directory, "rawdata/train_images.npy"))
    test_images = np.load(os.path.join(directory, "rawdata/test_images.npy"))
    train_labels = np.load(os.path.join(directory, "rawdata/train_labels.npy"))
    test_labels = np.load(os.path.join(directory, "rawdata/test_labels.npy"))

    tau = 3.e-3  # tau for penalty V~U_01
    pa = 5.e-6  # index pai for penalty U~WV
    gam = 1.e-5  # gamma for penalty norm2 "W"

    lr = 0.01  # 0.03 - 0.1   stepsize for choosing index of W
    sp0 = 1 - 0.7   # sparsity 0.6
    batch_size = len(train_labels)  # 批次大小
    # epochs = 20

    # normal parameters
    if 1:
        print(train_images.shape, batch_size)
        n_x = train_images.shape[1]  # 764--> 
        n_h1 = 2000  # 第一隐层
        n_h2 = 2000  # 第二隐层
        n_y = 10  # 输出层
        epoch_times = []  # 记录时间
        train_losses = []
        test_losses = []
        node1s = []
        node2s = []
        node3s = []
        precode = 0  # adam pretrain
        # adam parameters
        epsilon = 1.e-3  # adam步长的分子
        delta = 1.e-8  # adam步长的第一个分母
        rho_1 = 0.9  # adam 一阶距滑动系数
        rho_2 = 0.99  # adam 二阶距滑动系数
        step_ad = 10  # adam 二次下降
        # try:
        

(60000, 576) 60000


# **Pretrain steps to start**

In [ ]:
batch_size = 128
# require size: (60000, 1024) (10000, 1024) (60000, 10) (10000, 10)
n_x = train_images.shape[1]
parameters, _, _, g_sum = initialize_with_zeros(n_x, n_h1, n_h2, n_y)
prenum = 2
real_number = np.ceil(len(train_labels)/batch_size)
for i in range(prenum):
    # print('We are in the Ecope {}:'.format(i)) 
    # with tqdm(total = real_number) as pbar:
    train_data = mini_batchs_generator(train_images, train_labels, batch_size)
    for step, (img_train, label_train1) in enumerate(train_data):
        real_batch_size = len(label_train1)
        label_train = label_train1.T
        # imgvector1 = image2vector(img_train)
        imgvector = normalize_data(img_train.T)  # 784 *
        label_train = tf.convert_to_tensor(label_train, dtype=tf.float32)
        feat_train = tf.convert_to_tensor(imgvector, dtype=tf.float32)  # batch_size * feat nums  -->  feat nums * batch_size
        # print(feat_train.shape, parameters["W1"].shape)
        output, cache = forward_propagation_pre(feat_train, parameters)
        
        parameters, g_sum = optimizer_adam_l0(parameters, cache, feat_train, label_train, g_sum,
                                              epsilon, delta, sp0, rho_1, rho_2, step_ad)
save_parameters(parameters, os.path.join(directory, "preparameter"))

In [16]:
save_parameters(parameters, os.path.join(directory, "preparameter"))

# Load PreTraining data #

In [17]:
# input shape: (60000, 576) (10000, 576) (60000, 10) (10000, 10)
batch_size = 60000

trainall = normalize_data(train_images.T)
trainall = tf.convert_to_tensor(trainall, dtype=tf.float32)  # batch_size * feat nums  -->  feat nums * batch_size

testall = normalize_data(test_images.T)
testall = tf.convert_to_tensor(testall, dtype=tf.float32)       # batch_size * feat nums  -->  feat nums * batch_size

label_trainall = train_labels.T
label_trainall = tf.convert_to_tensor(label_trainall, dtype=tf.float32)

label_testall = test_labels.T
label_testall = tf.convert_to_tensor(label_testall, dtype=tf.float32)

In [ ]:
print(trainall.shape, testall.shape, label_trainall.shape, label_testall.shape)

(576, 60000) (576, 10000) (10, 60000) (10, 10000)


In [18]:
def optimizer(parameters, pinter, X, Y, epoc):
    def tenf(koa):
        koa = tf.convert_to_tensor(koa, dtype = tf.float32)
        return koa
    def subuout(ain, biin, tau_in):
        # ain batchsize * 10, bin too Cheaked at January 26th
        uf = biin
        row, column = np.nonzero(ain)
        biin = biin.numpy()
        cin = tf.transpose(tf.ones((10, batch_size)) * tf.convert_to_tensor(biin[row, column], dtype = tf.float32))
        uin2 = tf.minimum(biin, cin)
        condition1 = tf.sqrt(tf.convert_to_tensor(tau_in, dtype = tf.float32)) * tf.linalg.norm(biin - uin2, ord=2, axis=1)  # axis 1 represents row
        T1 = (condition1 < 2)
        uf = uf.numpy()
        uf[T1,:] = uin2.numpy()[T1,:]
        # uf = uin2.numpy()
        return tf.convert_to_tensor(uf.T, dtype = tf.float32)  # tf.transpose(uf)  #

    def subuin(ain, biin, uin, tau_in):
        epi = 3.e-7 
        u_p = biin.numpy()
        flag1 = u_p*0
        # kkk = np.zeros((10,2))
        # kkk[:,0] = ain.numpy()[0:10,0]
        # kkk[:,1] = biin.numpy()[0:10,0] 
        # print(kkk, 1 - tau_in * biin[0:10,0] * biin[0:10,0] / 2, 1 + tau_in * biin[0:10,0] * biin[0:10,0] / 2)
        condition1 = (biin > 0) & (ain < 0.5) & (ain <= (1 - tau_in * biin * biin) / 2)
        condition2 = (biin < 0) & (ain > 0.5) & (ain >= (1 + tau_in * biin * biin) / 2)
        flag1[condition1] = 1
        flag1[condition2] = 1  
        if sum(sum(flag1))==0:
          flag2 = 0
        else:
          flag2 = 1
          u_p[condition1] = 0
          u_p[condition2] = epi
          uin  = uin +  1 * (tf.convert_to_tensor(u_p, dtype = tf.float32) - uin)
        return uin, flag2

    def subw(win, bin, ain, tau_in, lr_in, gam_in, epoc, layer):
        # Win.shape n_{layer-1}, n_layer
        # A.shape n_{layer-1}, N
        # B.shape n_{layer}, N
        b = 2 * tau_in * tf.matmul(ain, tf.transpose(bin))
        A = 2 * tau_in * tf.matmul(ain, tf.transpose(ain)) + gam_in * tf.eye(len(b))
        d_W = tf.matmul(A, win) - b  # * gamma~~~~~~~~~~~~ 
        if layer > 1:
          if epoc < -2:
            win -= lr_in *  d_W
            idx_T = np.nonzero(tf.ones((win.shape[0], 1)))[0]
          else:
            if layer==2:
              win -= 0.01 * lr_in *  d_W
              idx_T = np.nonzero(tf.ones((win.shape[0], 1)))[0]
            else:
              W_try = win - lr_in *  d_W
              W_norm2 = prox_l0(W_try, sp0, vec_yn=0)
              idx_T = np.nonzero(W_norm2)[0]  # W行模的非0指标集Tk
              idx_Tbar = np.argwhere(W_norm2 == 0)[:, 0]
              win = win.numpy()
              ain = ain.numpy()
              b2 =  2 * tau_in * tf.matmul(tf.matmul(ain[idx_T], tf.transpose(ain[idx_Tbar])), win[idx_Tbar])
              b = tenf(b.numpy()[idx_T]) - b2
              # if epoc > 20:
              #   win[idx_Tbar] -= 1.e-2 * win[idx_Tbar]
                # if epoc > 30:
                #     win[idx_Tbar] = 0
              A1 = tf.linalg.inv(2 * tau_in * tf.matmul(tenf(ain[idx_T]), tenf(tf.transpose(ain[idx_T]))) + gam_in * tf.eye(len(idx_T)))
              win[idx_T] = tf.matmul(A1, b)
        else:
          if epoc % 6 < -10:
            win -= lr_in *  d_W
          else:
            win = tf.matmul(tf.linalg.inv(A), b)
          idx_T = np.nonzero(tf.ones((win.shape[0], 1)))[0]
        return tf.convert_to_tensor(win, dtype = tf.float32), idx_T  # tf.convert_to_tensor(, dtype = tf.float32)

    def subv(vin, idx, ain, bin, cin, pa_in, tau_in, layer):  # 显示解
        # print(len(idx), vin.shape)
        # vin: 1000x64;  ain:  1000[idx+1]x10;  bin:  10, 64;   cin: 1000[idx+1], 64,
        # for j in range(2):
        print(tau_in, pa_in)
        b = 2 * tau_in * tf.matmul(ain, bin) + 1000 * 2 * pa_in * cin
        A1 = tf.linalg.inv(2 * tau_in * tf.matmul(ain, tf.transpose(ain)) +  1000 * 2 * pa_in * tf.eye(len(b)))
        vin = tf.matmul(A1, b)
        return vin  # tf.convert_to_tensor(vin, dtype = tf.float32)

    for layer in range(3, 0, -1):
        if layer == 3:
            # Y 10*64;  W3 (1000, 10); V3 (10, 64);
            cost1be = pa[3] * tf.square( tf.norm(Y - hardmax(pinter["U3"])) ) + tau[3] * tf.square(tf.norm( pinter["U3"] - tf.matmul(tf.transpose(parameters['W3']),  pinter["V2"]))  )
            pinter['U' + str(layer)]= \
                subuout(tf.transpose(Y), tf.transpose(tf.matmul(tf.transpose(parameters["W" + str(layer)]), pinter['V' + str(layer - 1)])), tau[layer] * pa[layer])

            cost1af = pa[3] * tf.square( tf.norm(Y - hardmax(pinter["U3"])) ) + tau[3] * tf.square(tf.norm( pinter["U3"] - tf.matmul(tf.transpose(parameters['W3']),  pinter["V2"])))
            print("U3", cost1be.numpy(), cost1af.numpy())
            cost1be = tau[3] * tf.square(tf.norm( pinter["U3"] - tf.matmul(tf.transpose(parameters['W3']),  pinter["V2"]))) + gam/2 * (tf.square( tf.norm(parameters['W3'])))
            parameters['W' + str(layer)], parameters["idx" + str(layer)] = \
                subw(parameters['W' + str(layer)], pinter['U' + str(layer)], pinter['V' + str(layer - 1)], tau[layer], lr, gam, epoc, layer)
            cost2af = tau[3] * tf.square(tf.norm( pinter["U3"] - tf.matmul(tf.transpose(parameters['W3']),  pinter["V2"]))) + gam/2 * (tf.square( tf.norm(parameters['W3'])))
            print("W3",cost1be.numpy(), cost2af.numpy())

        else:
            costvbe = tau[layer+1] * tf.square(tf.norm(pinter['U' + str(layer + 1)] - tf.matmul(tf.transpose(parameters["W" + str(layer + 1)]), pinter['V' + str(layer)]))) + pa[layer] * tf.square(tf.norm(pinter['V' + str(layer)] - sgn(pinter['U' + str(layer)])))
            print((tau[layer+1] * tf.square(tf.norm(pinter['U' + str(layer + 1)] - tf.matmul(tf.transpose(parameters["W" + str(layer + 1)]), pinter['V' + str(layer)])))).numpy(),
                  (pa[layer] * tf.square(tf.norm(pinter['V' + str(layer)] - sgn(pinter['U' + str(layer)])))).numpy())
            pinter['V' + str(layer)] = subv(pinter['V' + str(layer)], parameters["idx" + str(layer + 1)],
                                            parameters["W" + str(layer + 1)], pinter['U' + str(layer + 1)],
                                            sgn(pinter['U' + str(layer)]), pa[layer], tau[layer+1], layer)  # , parameters["idx" + str(layer + 1)])
            costvaf = tau[layer+1] * tf.square(tf.norm(pinter['U' + str(layer + 1)] - tf.matmul(tf.transpose(parameters["W" + str(layer + 1)]), pinter['V' + str(layer)]))) + pa[layer] * tf.square(tf.norm(pinter['V' + str(layer)] - sgn(pinter['U' + str(layer)])))
            
            print((tau[layer+1] * tf.square(tf.norm(pinter['U' + str(layer + 1)] - tf.matmul(tf.transpose(parameters["W" + str(layer + 1)]), pinter['V' + str(layer)])))).numpy(),
                   (pa[layer] * tf.square(tf.norm(pinter['V' + str(layer)] - sgn(pinter['U' + str(layer)])))).numpy())
            
            print("V"+ str(layer), costvbe.numpy(), costvaf.numpy())

            costubf = pa[layer] * tf.square( tf.norm(pinter['V' + str(layer)] - sgn(pinter['U' + str(layer)]))) + tau[layer] * tf.square(tf.norm( pinter['U' + str(layer)] - tf.matmul(tf.transpose(parameters["W" + str(layer)]),  pinter['V' + str(layer - 1)])))
            pinter['U' + str(layer)], flag2 = subuin(pinter['V' + str(layer)],
                                              tf.matmul(tf.transpose(parameters["W" + str(layer)]), pinter['V' + str(layer - 1)]),
                                              pinter['U' + str(layer)],  tau[layer] / pa[layer])
            costuaf = pa[layer] * tf.square( tf.norm(pinter['V' + str(layer)] - sgn(pinter['U' + str(layer)]))) + tau[layer] * tf.square(tf.norm( pinter['U' + str(layer)] - tf.matmul(tf.transpose(parameters["W" + str(layer)]),  pinter['V' + str(layer - 1)])))
            print("Uin"+ str(layer), costubf.numpy(), costuaf.numpy())

            if flag2:
              costwinbf = tau[layer] * tf.square(tf.norm( pinter['U' + str(layer)] - tf.matmul(tf.transpose(parameters['W' + str(layer)]),   pinter['V' + str(layer - 1)]))) + gam/2 * (tf.square( tf.norm(parameters['W' + str(layer)])))
              parameters['W' + str(layer)], parameters["idx" + str(layer)] = \
                  subw(parameters['W' + str(layer)], pinter['U' + str(layer)], pinter['V' + str(layer - 1)], tau[layer], lr,  gam, epoc, layer)
              costwinaf = tau[layer] * tf.square(tf.norm( pinter['U' + str(layer)] - tf.matmul(tf.transpose(parameters['W' + str(layer)]),   pinter['V' + str(layer - 1)]))) + gam/2 * (tf.square( tf.norm(parameters['W' + str(layer)])))
              print("Win"+ str(layer), costwinbf.numpy(), costwinaf.numpy())

            # cost3 = pa[2] * tf.square( tf.norm(C["V2"] - sgn(C["U2"])) ) + tau[2] * tf.square( tf.norm(C["U2"] - tf.matmul(tf.transpose(P['W2']), C["V1"])) )
            # print("Cheack out, now-------------------")
            # print(pa[2], tf.square( tf.norm(C["V2"] - sgn(C["U2"])) ), tau[2], tf.square( tf.norm(C["U2"] - tf.matmul(tf.transpose(P['W2']), C["V1"])) ))
            # print("------------------------------------")
            # cost4 =  pa[1] * tf.square( tf.norm(C["V1"] - sgn(C["U1"])) ) + tau[1] * tf.square( tf.norm(C["U1"] - tf.matmul(tf.transpose(P['W1']), X)) )
            # cost5 = gam * (tf.square( tf.norm(P['W3'])) + tf.square(tf.norm(P['W2'])) + tf.square(tf.norm(P['W1'])))
            
    return parameters, pinter

# **3 Train 0/1 DNN by PBCD**

In [19]:
label_train = label_trainall
label_test = label_testall

batch_size = 60000
history1 = np.zeros((100,1))
history2 = np.zeros((100,1))
super1_all = [7.2]  # current best parameter 1.e-4:
epochs = 40
low_acc = -8000
best_acc_of_all = 0  # 1.2
best_para1 = 0.0072  # 0.3
for super1 in super1_all:
    sp0 = 1 - 0.45   # sparsity 0.7
    lr = 7.2e-4  # 6.e-2 ~ 6.e-1  turned at Feb 17th
    ship = 2.e-2  # slippage for the next step
    gam = 1.e-6  # 1.e-9  # turned at Feb 27th 6.e-2  --->  6.e-3   gamma for penalty norm2 "W"
    best_acc = 0
    parameters = load_parameters(os.path.join(directory, "preparameter"))
    # parameters, pa, tau, _ = initialize_with_zeros(n_x, n_h1, n_h2, n_y)

    pa = 1.e-5 * np.ones((4, 1))          # 每一层罚参数初始值
    pa[3] = 1 * 1000 / (2 * batch_size)   # 每一层罚参数初始值 8.33333333e-06 
    tau = 1.e-4 * np.ones((4, 1))         # 每一层罚参数初始值
    pa[2] = 1.e-4

    # try:   
    for i in range(epochs):
        print("-------------------------------------------")
        # aa25, _ = forward_propagation(trainall, parameters)
        # acc = (60000 - costloss(aa25, label_train))/6
        # history1[i] = acc
        
        if i ==4 or (i % 5 == 0 and i>8 and i<31) or (i % 6 == 0 and i>35): 
          _, _, _, g_sum = initialize_with_zeros(n_x, n_h1, n_h2, n_y)
          train_data_in = mini_batchs_generator(trainall.numpy().T, label_train.numpy().T, 256)
          for step, (img_train_in, label_train1_in) in enumerate(train_data_in):
              real_batch_size = len(label_train1_in)
              label_train_in = label_train1_in.T
              label_train_in = tf.convert_to_tensor(label_train_in, dtype=tf.float32)
              feat_train_in = tf.convert_to_tensor(img_train_in.T, dtype=tf.float32)  # batch_size * feat nums  -->  feat nums * batch_size
              # print(feat_train.shape, parameters["W1"].shape)
              output, cache1 = forward_propagation_pre(feat_train_in, parameters)
              
              parameters, g_sum = optimizer_adam_l0(parameters, cache1, feat_train_in, label_train_in, g_sum,
                                                      epsilon, delta, sp0, rho_1, rho_2, step_ad)
        else:
          if i % 2 == 0:
            aa21, cache = forward_propagation(trainall, parameters)
          parameters, cache = optimizer(parameters, cache, trainall, label_train, i)
          
        if i % 1 == 0:
          merit = penaltyloss(parameters, cache, trainall, label_train)
          # acc_train = costloss(aa21, label_train)
          print("After", merit["cost1"], merit["cost2"], merit["cost3"], merit["cost4"], 
                merit["cost5"], merit["costall"])
        if merit["cost1"] < 0.7 * merit["cost5"]:
          gam *= 0.5
        # if i % 10 == 0 and i > 2:
        #   lr *= 0.8

        aa25, _ = forward_propagation(testall, parameters)
        acc = 10000 - costloss(aa25, label_test)
        history2[i] = acc

        if acc < low_acc or merit["costall"] > 10000:
            print("Too low acc:{} or merit:{}.".format(acc, merit["costall"]))
            break
        else:
            print('Famal train: The :{}-th epoch, current super1:{}. and the current accuracy in this step: {}'.format(
                    i, super1, acc))
        if acc > best_acc:
            best_acc = acc
            print('Updating best accuracy in this epoch:{}'.format(best_acc))

    if best_acc > best_acc_of_all:
        best_acc_of_all = best_acc
        best_para1 = super1
        print("========================================================")
        print("=======================New accuracy=====================")
        print("The current best acc of all is: {}, super parameter1 is: {},".format(
            best_acc_of_all, best_para1))
        print("========================================================")
    # except:
    #     print("There are some wrong things, but pass.")
    #     continue   # break
print("The final best acc of all is: {}, super parameter1 is: {}:".format(best_acc_of_all, best_para1))

-------------------------------------------
U3 [99.200005] [73.5498]
W3 [11.533136] [9.27671]
[9.276692] [0.]
[0.0001] [0.0001]
[9.185942] [4.5247652e-05]
V2 [9.276692] [9.1859865]
Uin2 [4.5247652e-05] [4.5247652e-05]
[0.] [0.]
[0.0001] [1.e-05]
[3.552563e-10] [1.6776636e-11]
V1 [0.] [3.7203293e-10]
Uin1 [1.6776636e-11] [1.6776636e-11]
Cheack out, now-------------------
[0.0001] 0.45247653 [0.0001] 3.552563e-06
------------------------------------
After [62.016674] [9.185942] [4.524801e-05] [1.6776636e-11] 0.0033593269 [71.20602]
Famal train: The :0-th epoch, current super1:7.2. and the current accuracy in this step: 8926.0
Updating best accuracy in this epoch:8926.0
-------------------------------------------
U3 [71.202614] [66.99572]
W3 [7.945735] [5.5809903]
[4.819643] [4.5247652e-05]
[0.0001] [0.0001]
[0.0309345] [0.00010619]
V2 [4.8196883] [0.0310407]
Uin2 [0.00010619] [0.00010619]
Win2 [0.00114483] [0.00114483]
[2.5673313e-14] [1.6776636e-11]
[0.0001] [1.e-05]
[3.723578e-11] [1.7

In [ ]:
save_parameters(parameters, os.path.join(directory, "finalparameter"))

In [ ]:
np.save(os.path.join(direction, "result/history1.npy"), history1)
np.save(os.path.join(direction, "result/history2.npy"), history2)

In [ ]:
import tensorflow as tf
import torch as tc
import numpy as np
from tqdm import tqdm, trange
import time

import time
from tqdm import tqdm, trange


a1 = tf.random.uniform([2,3], minval=-1, maxval=1, dtype=tf.dtypes.float32, seed=None, name=None)
a1 = a1.numpy()
a1[:,:] = 0
condition1 = (a1 < 0) 
a1[condition1] = 999
print(condition1, a1)

[[False False False]
 [False False False]] [[0. 0. 0.]
 [0. 0. 0.]]
